In [ ]:
import pandas as pd
import time
import io

from api_request_funtions import ApiGet
from bq_transfers import BqDataTransfers 
# from main import fetch_and_save
import json

In [ ]:
thrivecart_get = ApiGet(http='thrivecart.com', api_key='TZ5TJYBR-FDB85IBI-0RFTB00N-VQ7ZFY2S')
thrivecart_save = BqDataTransfers(gcp_project_id= 'arboreal-cat-451816-n0', bq_data_set = 'thrive_cart')
bq_client = thrivecart_save.get_bq_client("/Users/shami/Library/Mobile Documents/com~apple~CloudDocs/Personal Projects/vexis/vexis_bq_writter.json")

# split products

In [11]:
import requests
import json

# Define your API URL and credentials
api_url = "https://api.kajabi.com/v1/payments/transactions"  # Replace with the correct API endpoint
api_token = 'xxxxxxxx'
api_secret = 'xxxxxxxxxxxxx' # Replace with your API secret if required

# Set the parameters for the query
params = {
    "page": 1,
    "sort": "date",
    "direction": "desc",
    "in_the_last": "30_days"

headers = {
    "Authorization": f"Bearer {api_token}",
    "Content-Type": "application/json"
}

response = requests.get(api_url, headers=headers, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))  # Pretty print the JSON data
else:
    # Handle errors
    print(f"Error: {response.status_code}")
    print(response.text)


Error: 404
{"message":"Not Found"}


In [1]:
import requests

token_url = 'https://kajabi.com/oauth/token'

data = {
  "client_id": "xxxxxxxxxxx",
  "client_secret": "xxxxxxxxxxx",
  "grant_type": "client_credentials"
}


response = requests.post(token_url, data=data)
access_token = response.json()['access_token']

/Users/shami/personal_projects/vexis_authLive/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
import requests

url = "https://app.kajabi.com/api/v1/me"
data = {
    "username": "authenticliving@vexis.co.uk",
    "password": "Authenticliving@3141"
}

response = requests.post(url, json=data)

# Print the response (access_token and refresh_token)
print(response.json())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
from api_request_funtions import ApiGet
from api_request_funtions import ApiGetRequest
from bq_transfers import BqDataTransfers
from  bq_transfers import pub_sub_message_publisher
import io
import time
import numpy as np
from datetime import datetime
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import pandas as pd
import json
from urllib.parse import urlencode

/Users/shami/personal_projects/vexis_authLive/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
project_id = 'arboreal-cat-451816-n0'
headers = {
    'API-Key': 'API_c1c14fe7384d5050491b9d0c401184bd6facb348ccc8c59d2a53cdcec9c14332'
}
hyros_save = BqDataTransfers(gcp_project_id=project_id, bq_data_set='hyros')
bq_client = hyros_save.get_bq_client(
    "/Users/shami/Library/Mobile Documents/com~apple~CloudDocs/Personal Projects/vexis/vexis_bq_writter.json"
)

def _process_and_save_df(df: pd.DataFrame, table_id: str, date, page_id):
    """
    Aligns and uploads the DataFrame to BigQuery.
    Adds missing columns to table or DataFrame to ensure schema match.
    """
    print(f"🔄 Processing data for {table_id} | Date: {date} | Page: {page_id}")
    try:
        df['creationDate'] = pd.to_datetime(df['creationDate']).dt.tz_convert('UTC')
    except:
        df['creationDate'] = pd.to_datetime(df['creationDate'], unit='ms', utc=True)
    try:
        df.columns = df.columns.str.replace(r'\.', '_', regex=True)
        align_and_upload_to_bq(df, f"hyros.{table_id}", project_id=hyros_save.gcp_project_id)
    except Exception as e:
        print(f"Failed to process and upload data: {e}")


def align_and_upload_to_bq(df, table_id, project_id):
    # Flatten list-like columns (arrays) into strings to prevent pyarrow issues
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list)).any():
            print(f"⚠️ Flattening list column: {col}")
            df[col] = df[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)

    # Get current table schema
    table = bq_client.get_table(table_id)
    bq_schema = table.schema

    # Create sets for comparison
    df_columns = set(df.columns)
    bq_columns = set(field.name for field in bq_schema)

    # Columns in df but not in BigQuery table (need to add to BQ table)
    columns_to_add_to_bq = df_columns - bq_columns

    for col in columns_to_add_to_bq:
        dtype = df[col].dropna().infer_objects().dtype
        if pd.api.types.is_string_dtype(dtype):
            field_type = "STRING"
        elif pd.api.types.is_bool_dtype(dtype):
            field_type = "BOOLEAN"
        elif pd.api.types.is_integer_dtype(dtype):
            field_type = "INT64"
        elif pd.api.types.is_float_dtype(dtype):
            field_type = "FLOAT64"
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            field_type = "TIMESTAMP"
        else:
            field_type = "STRING"  # fallback for complex types

        query = f"""
        ALTER TABLE `{table_id}`
        ADD COLUMN `{col}` {field_type};
        """
        bq_client.query(query).result()
        print(f"➕ Added column `{col}` ({field_type}) to {table_id}")

    # Columns in BQ table but not in df (need to add as nulls)
    columns_to_add_to_df = bq_columns - df_columns
    for col in columns_to_add_to_df:
        dtype = next(field.field_type for field in bq_schema if field.name == col)
        if dtype == "STRING":
            df[col] = None
        elif dtype in ["INT64", "FLOAT64", "BOOLEAN"]:
            df[col] = np.nan
        elif dtype == "TIMESTAMP":
            df[col] = pd.NaT
        else:
            df[col] = None  # fallback

    # Reorder columns to match BQ schema
    df = df[[field.name for field in bq_schema]]

    # Upload to BigQuery
    job = bq_client.load_table_from_dataframe(df, table_id)
    job.result()  # Wait for completion
    print(f"✅ Uploaded {len(df)} rows to {table_id}")

def fetch_and_store_hyros_data(
    endpoint: str,
    destination_table: str,
    start_date: str = None,
    end_date: str = None,
    use_date_and_pagination: bool = True,
):
    resume_table = f"{destination_table}_resume_state"
    dataset_id = hyros_save.bq_data_set
    full_table_id = f"{hyros_save.gcp_project_id}.{dataset_id}.{resume_table}"

    def get_resume_state():
        try:
            query = f"SELECT page_id FROM `{full_table_id}` LIMIT 1"
            result = bq_client.query(query).result()
            row = next(result, None)
            if row:
                return int(row.page_id)
        except Exception as e:
            print(f"Couldn't read resume state from BQ: {e}")
        return 1  # default page_id

    def save_resume_state(page_id):
        df = pd.DataFrame([{"page_id": int(page_id)}])
        hyros_save.start_transfer_df(
            bq_client=bq_client,
            df=df,
            destination_table=resume_table,
            write_options='overwrite'
        )
        print(f"Saved resume state: page_id={page_id}")

    if use_date_and_pagination:
        if not start_date or not end_date:
            raise ValueError("Both 'start_date' and 'end_date' are required when using pagination.")

        # Convert to full-day datetime strings
        start_dt = datetime.fromisoformat(start_date).replace(hour=0, minute=0, second=0, microsecond=0)
        end_dt = datetime.fromisoformat(end_date).replace(hour=23, minute=59, second=59, microsecond=0)
        fromDate = start_dt.isoformat()
        toDate = end_dt.isoformat()

        page_id = get_resume_state()
        page_counter = 0
        collected_pages = []

        while True:
            url = f"https://api.hyros.com/v1/api/v1.0/{endpoint}?fromDate={fromDate}&toDate={toDate}&pageSize=250&pageId={page_id}"
            print(f"Request: {url}")
            request = Request(url, headers=headers)

            try:
                response_body = urlopen(request).read()
                json_str = response_body.decode('utf-8')
                data = json.loads(json_str)
                page_df = pd.json_normalize(data['result'])

                if page_df.empty:
                    print(f"No data for date range {start_date} to {end_date}, page {page_id}")
                    break

                collected_pages.append(page_df)
                page_counter += 1
                next_page = data.get('nextPageId')

                # Process & save every 10 pages
                if page_counter % 10 == 0:
                    combined_df = pd.concat(collected_pages, ignore_index=True)
                    _process_and_save_df(combined_df, destination_table, start_date, page_id)
                    collected_pages = []
                    save_resume_state(next_page)

                # If no more pages, process remaining data and save resume
                if not next_page:
                    if collected_pages:
                        combined_df = pd.concat(collected_pages, ignore_index=True)
                        _process_and_save_df(combined_df, destination_table, start_date, page_id)
                    save_resume_state(1)  # reset tracker
                    break

                page_id = next_page

            except Exception as e:
                print(f"Error on page {page_id}: {e}")
                return

    else:
        print(f"Fetching non-date endpoint: {endpoint}")
        url = f"https://api.hyros.com/v1/api/v1.0/{endpoint}"
        request = Request(url, headers=headers)

        try:
            response_body = urlopen(request).read()
            json_str = response_body.decode('utf-8')
            data = json.loads(json_str)
            df = pd.json_normalize(data['result'])

            if df.empty:
                print("No data returned.")
            else:
                _process_and_save_df(df, destination_table, "n/a", "n/a")
        except Exception as e:
            print(f"API call failed: {e}")

In [4]:
query = """
    SELECT DISTINCT lead.firstSource.adSource.adSourceId AS adSourceId
    FROM `arboreal-cat-451816-n0.hyros.user_journey`
    WHERE lead.firstSource.adSource.adSourceId IS NOT NULL
"""
ad_source_ids_df = bq_client.query(query).to_dataframe()
ad_source_ids = ad_source_ids_df['adSourceId'].dropna().astype(str).tolist()


/Users/shami/personal_projects/vexis_authLive/.venv/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [8]:
ad_source_ids_param = ",".join(f'"{x}"' for x in ad_source_ids[0:9])
include_organic = "true"
include_disregarded = "false"
integration_type = "ALL"
page_size = 250
page_id = 1

In [9]:
endpoint = f"""sources?adSourceIds={ad_source_ids_param}&includeOrganic={include_organic}&includeDisregarded={include_disregarded}&integrationType={integration_type}&pageSize={page_size}&pageId={page_id}"""

# Call the function (non-date based)
fetch_and_store_hyros_data(
    endpoint=endpoint,
    destination_table='sources',
    use_date_and_pagination=False  # because this endpoint is not date-based
)

Fetching non-date endpoint: sources?adSourceIds="120228331837710196","120229270339050196","120229964872380242","120228028354760283","120227927598160283","120227767691450283","120229068614010196","120229068741670196","120225673914670578"&includeOrganic=true&includeDisregarded=false&integrationType=ALL&pageSize=250&pageId=1
API call failed: HTTP Error 400: 
